<a href="https://colab.research.google.com/github/MichaelTj02/FinalProject_Group1/blob/main/LicensePlate_CRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract license plate characters using CRNN

Dataset loader

In [40]:
# Create dataset.py file using this code
import os
from PIL import Image
import torch
from torch.utils.data import Dataset

class LicensePlateDataset(Dataset):
    def __init__(self, label_file, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.data = []

        # Read the label file and parse it
        with open(label_file, "r") as f:
            for line in f:
                parts = line.strip().split()
                image_name = parts[0]  # Image name
                text = parts[1]        # Label text
                self.data.append((image_name, text))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_name, text = self.data[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        # Convert text to tensor (optional: depends on your CRNN implementation)
        label = torch.tensor([ord(c) for c in text], dtype=torch.long)  # Example: ASCII encoding

        return image, label



Define CRNN model

In [11]:
# Create model.py file using this code
import torch
import torch.nn as nn

class CRNN(nn.Module):
    def __init__(self, num_classes, input_dim=1, hidden_dim=256):
        super(CRNN, self).__init__()
        # CNN layers
        self.cnn = nn.Sequential(
            nn.Conv2d(input_dim, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # RNN layers
        self.rnn = nn.LSTM(128 * 8 * 8, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        x = self.cnn(x)  # Apply CNN
        x = x.view(x.size(0), -1, 128)  # Reshape for LSTM
        x, _ = self.rnn(x)  # Apply LSTM
        x = self.fc(x[:, -1, :])  # Fully connected layer
        return x


# Train CRNN with the dataset

In [7]:
# Setup paths and imports
import os
import torch

import sys
sys.path.append(r"C:\Users\mtj02\Desktop\FinalProject_Group1\Datasets\LicensePlateDataset")
from dataset import LicensePlateDataset

from model import CRNN
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Resize, ToTensor

# Define paths
project_dir = r"C:\Users\mtj02\Desktop\FinalProject_Group1\Datasets\LicensePlateDataset"

In [8]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    images, labels = zip(*batch)

    # Stack images into a tensor
    images = torch.stack(images)

    # Convert labels to tensors and pad them
    labels = pad_sequence(labels, batch_first=True, padding_value=0)  # Use 0 as the padding value

    return images, labels

In [9]:
from torchvision import transforms
from torch.utils.data import DataLoader

# Define the paths to the dataset and labels
train_image_dir = r"C:\Users\mtj02\Desktop\FinalProject_Group1\Datasets\LicensePlateDataset\train\images"
train_label_file = r"C:\Users\mtj02\Desktop\FinalProject_Group1\Datasets\LicensePlateDataset\train_labels.txt"
val_image_dir = r"C:\Users\mtj02\Desktop\FinalProject_Group1\Datasets\LicensePlateDataset\val\images"
val_label_file = r"C:\Users\mtj02\Desktop\FinalProject_Group1\Datasets\LicensePlateDataset\val_labels.txt"

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((32, 128)),  # Resize to fixed dimensions
    transforms.ToTensor(),        # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the image
])

# Load the datasets
train_dataset = LicensePlateDataset(train_label_file, train_image_dir, transform=transform)
val_dataset = LicensePlateDataset(val_label_file, val_image_dir, transform=transform)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

# Test the first batch from the DataLoader
for images, labels in train_loader:
    print(f"Batch image shape: {images.shape}")
    print(f"Batch labels shape: {labels.shape}")
    break


Training dataset size: 167
Validation dataset size: 42
Batch image shape: torch.Size([32, 3, 32, 128])
Batch labels shape: torch.Size([32, 11])
